In [3]:
#Slef-made Function
from Dataload import dataload
from module import transformer,video_model
from constant import EMOTIPATH,EMOTIFACEPATH,EMOTIAUDIOPATH
from train import methods


#Torch Library

import torch
import torch.nn as nn
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torchvision import  utils

#Sub tools
import pandas as pd

import os
import matplotlib.pylab as plt

#Util Library
import time
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1 import ImageGrid  

Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_video_pt=os.path.join(EMOTIPATH,"pt","Train")
Train_face_pt=os.path.join(EMOTIFACEPATH,"pt_stacked","Train")
Audio_Train=os.path.join(EMOTIAUDIOPATH,"Train_audio.csv")

Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Val_video_pt=os.path.join(EMOTIPATH,"pt","Valid")
Val_face_pt=os.path.join(EMOTIFACEPATH,"pt_stacked","Valid")
Audio_Valid=os.path.join(EMOTIAUDIOPATH,"Val_audio.csv")


train_table = pd.read_csv(Train_label,delimiter=' ')
val_table = pd.read_csv(Val_labels,delimiter=' ')
print(train_table['Label'].value_counts())
print(val_table['Label'].value_counts())



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)



frame_num=10

train_data_pt=dataload.Video_Frame_Data(Train_label,base_path_v=Train_video_pt,face_path=Train_face_pt,frame_num=frame_num,direct=True,audio_csv=Audio_Train)
valid_data_pt=dataload.Video_Frame_Data(Val_labels,base_path_v=Val_video_pt,face_path=Val_face_pt,frame_num=frame_num,direct=True,audio_csv=Audio_Valid)





def load_pretrained_model(pre_train):
    
    face_model=torch.load("pre_trained_model/" +"pre_trained_resnet18_face.h")
    audio_model=video_model.AudioRecognition(softmax=pre_train)
    audio_model.load_state_dict(torch.load("pre_trained_model/pre_embedded_audio.pth"),strict=False)
    image_model=video_model.Video_modeller(frame_num,face_model=face_model,pre_train=pre_train)
    image_model.load_state_dict(torch.load("pre_trained_model/img_global_5.pth"),strict=False)
    return(image_model,audio_model)



def show_img(dataset,index,frame):
    x=dataset[index][0][frame].cpu().numpy()
    plt.figure(1,(15,15))
    plt.axis('off')
    image = (x*0.5+0.5).transpose((1, 2, 0)).squeeze()
    plt.imshow(image)
    
    fig = plt.figure(2,(15,15))
   # plt.axis('off')
    grid = ImageGrid(fig, 111,
                     nrows_ncols=(1,5),
                     axes_pad=0.1,
                     )
    for i in range(5):
        face=dataset[index][1][frame].cpu().numpy()
        image = (face[i,:]*0.5+0.5).transpose((1, 2, 0)).squeeze()
        grid[i].imshow(image,cmap='gray',interpolation='none')
        
img_model,audio_model=load_pretrained_model(False)



3    932
2    923
1    806
Name: Label, dtype: int64
1    299
2    281
3    186
Name: Label, dtype: int64


In [5]:
face_model=torch.load("pre_trained_model/" +"pre_trained_resnet18_face.h")

In [6]:
face_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:


weights=[932/932,2,932/806]
class_weights = torch.FloatTensor(weights).to(device)

train_dataloader = DataLoader(train_data_pt, batch_size=32
                       , num_workers=0,shuffle=True)
valid_dataloader = DataLoader(valid_data_pt, batch_size=32
                   , num_workers=0)
model=video_model.video_transformer(img_model,audio_model)
model=model.to(device)

num_epochs=50
criterion = nn.CrossEntropyLoss(weight=class_weights)

for name, child in model.named_children():
    if not name in ['face_model','frame_model' ]:
        print(name + ' is unfrozen')
        for param in child.parameters():
            param.requires_grad = True
    else:
        print(name + ' is frozen')
        for param in child.parameters():
               param.requires_grad = False

In [3]:

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0005, betas=(0.5, 0.999))


model_name="v7"

print(model_name)

methods.full_train("v7_model",num_epochs,model_name,model,train_dataloader,valid_dataloader,optimizer,criterion,device)


v7
Epoch:  1 

Validation 

Validation Loss:  1.0793790618578594
Validation Accuracy:  0.3838120104438642
Confusion Matrix: 
 [[140  47 112]
 [187  33  61]
 [ 60   5 121]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.9089927673339844
Batch:  80 / 84
Batch Recognition loss:  0.9783446192741394

Average_Loss:  0.8549933163892656
Average_Accuracy:  0.6816986095452837
Confusion Matrix: 
 [[423 267 116]
 [ 91 714 118]
 [ 70 185 677]]
Epoch:  2 

Validation 

Validation Loss:  1.0159535308678944
Validation Accuracy:  0.5378590078328982
Confusion Matrix: 
 [[213  51  35]
 [138 123  20]
 [ 59  51  76]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.8722290396690369
Batch:  80 / 84
Batch Recognition loss:  0.9121275544166565

Average_Loss:  0.8329844155481884
Average_Accuracy:  0.6952273581360391
Confusion Matrix: 
 [[446 252 108]
 [ 86 726 111]
 [ 78 176 678]]
Epoch:  3 

Validation 

Validation Loss:  1.0291012525558472
Validation Accuracy:  0.5143603133159269
Confusion Matrix: 
 [[210  66  23]
 [124 132  25]
 [ 57  77  52]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.795126736164093
Batch:  80 / 84
Batch Recognition loss:  0.9640493392944336

Average_Loss:  0.8147768427928289
Average_Accuracy:  0.7095077038707253
Confusion Matrix: 
 [[465 235 106]
 [ 86 753  84]
 [ 79 183 670]]
Epoch:  4 

Validation 

Validation Loss:  0.9975238665938377
Validation Accuracy:  0.49738903394255873
Confusion Matrix: 
 [[ 77 155  67]
 [ 40 207  34]
 [ 13  76  97]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.7508949041366577
Batch:  80 / 84
Batch Recognition loss:  0.8474494814872742

Average_Loss:  0.8087333731708073
Average_Accuracy:  0.7158962795941376
Confusion Matrix: 
 [[467 245  94]
 [ 94 741  88]
 [ 64 171 697]]
Epoch:  5 

Validation 

Validation Loss:  1.0026092330614726
Validation Accuracy:  0.4960835509138381
Confusion Matrix: 
 [[107 155  37]
 [ 53 205  23]
 [ 36  82  68]]
Train 



...

Batch:  40 / 84
Batch Recognition loss:  0.6563746333122253
Batch:  80 / 84
Batch Recognition loss:  0.7597068548202515

Average_Loss:  0.7299504358144033
Average_Accuracy:  0.7996993611424277
Confusion Matrix: 
 [[561 175  70]
 [ 53 820  50]
 [ 34 151 747]]
Epoch:  47 

Validation 

Validation Loss:  1.026978721221288
Validation Accuracy:  0.5234986945169713
Confusion Matrix: 
 [[150  76  73]
 [ 88 143  50]
 [ 50  28 108]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.7041772603988647
Batch:  80 / 84
Batch Recognition loss:  0.7278287410736084

Average_Loss:  0.7508907743862697
Average_Accuracy:  0.7820368282600526
Confusion Matrix: 
 [[539 193  74]
 [ 70 798  55]
 [ 60 128 744]]
Epoch:  48 

Validation 

Validation Loss:  1.0514092420538266
Validation Accuracy:  0.46736292428198434
Confusion Matrix: 
 [[ 86 116  97]
 [ 43 161  77]
 [ 27  48 111]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.7148075103759766
Batch:  80 / 84
Batch Recognition loss:  0.7559669017791748

Average_Loss:  0.7314229025727227
Average_Accuracy:  0.7978203682826005
Confusion Matrix: 
 [[534 188  84]
 [ 44 818  61]
 [ 39 122 771]]
Epoch:  49 

Validation 

Validation Loss:  1.0628923624753952
Validation Accuracy:  0.45691906005221933
Confusion Matrix: 
 [[115 116  68]
 [ 78 165  38]
 [ 52  64  70]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.842606246471405
Batch:  80 / 84
Batch Recognition loss:  0.6719943284988403

Average_Loss:  0.732770534498351
Average_Accuracy:  0.7974445697106352
Confusion Matrix: 
 [[558 182  66]
 [ 52 821  50]
 [ 48 141 743]]
Epoch:  50 

Validation 

Validation Loss:  1.0283719102541606
Validation Accuracy:  0.47780678851174935
Confusion Matrix: 
 [[ 64 146  89]
 [ 29 197  55]
 [ 28  53 105]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.6450212597846985
Batch:  80 / 84
Batch Recognition loss:  0.7017750144004822

Average_Loss:  0.7332355883859453
Average_Accuracy:  0.8023299511461857
Confusion Matrix: 
 [[565 163  78]
 [ 54 812  57]
 [ 44 130 758]]
